In [48]:
from trp import Document
import boto3

# Document
documentName = "C:/Users/hrint/Documents/Python2022/healthcare/extraction/Referral_Form_page-0001.jpg"
# Amazon Textract client
textract = boto3.client('textract')
# Call Amazon Textract
with open(documentName, "rb") as document:
    response = textract.analyze_document(
        Document={
            'Bytes': document.read(),
        },
        FeatureTypes=["FORMS",'TABLES' ])

# Get the text blocks
blocks=response['Blocks']
doc = Document(response)

c:\users\hrint\appdata\local\programs\python\python36\lib\site-packages\boto3\compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [50]:
for field in doc.pages[0].form.fields:
    print(field.key, "------" ,field.value)

Phone: ------ (432) 432-4233
Referred On: ------ 01/07/2022
Phone: ------ (234)324-3244
City, St Zip: ------ HICKORY, NC 28602-4031
City, St Zip: ------ Naples, FL 34109
Referred To: ------ BROWN, VIRGINIA
Fax: ------ (234) 234-2343
Policy#: ------ 4555533
Address: ------ 2406 CENTURY PL SE
DOB: ------ 10-02-1972
City, St Zip: ------ Fort Lauderdale, FL 33305
Address: ------ 2430 Vanderbilt Beach Road, Suite 355
Referred By: ------ Family Frank
Patient Name: ------ Sparling, Michael
Phone: ------ (828) 324-9595
Fax: ------ (828) 324-4154
Primary/Billing Diagnosis: ------ /27.21 Secondary pulmonary arterial hypertension
Address: ------ 3128 Rocket Drive
Insurance: ------ Affinity Health Plan Medicare
Referral Reason: ------ None
Attachments: ------ Unsigned Chart Note (Jan 07, 2022 04:00 PM) (Unsigned Chart Note (Jan 07, 2022 04:00 PM))  Michael_Sparling_1304168_202217142458.xml (Michael_Sparling_1304168_202217142458)


In [110]:
for block in blocks:
#     if block['BlockType'] == 'CELL':
#             print("    Cell information")
#             print("        Column: " + str(block['ColumnIndex']))
#             print("        Row: " + str(block['RowIndex']))
#             print("        ColumnSpan: " + str(block['ColumnSpan']))
#             print("        RowSpan: " + str(block['RowSpan'])) 
# #     if 'Relationships' in block:
#         print('    Relationships: {}'.format(block['Relationships']))
#         print('    Geometry: ')
#         print('        Bounding Box: {}'.format(block['Geometry']['BoundingBox']))
#         print('        Polygon: {}'.format(block['Geometry']['Polygon']))
    
    if block['BlockType'] == "KEY_VALUE_SET":
#         print((block['EntityTypes'][0]))
#         print ('    Entity Type: ' + block['EntityTypes'][0])
    
        print()

SyntaxError: invalid syntax (<ipython-input-110-4d945da50b36>, line 18)

In [75]:
from PIL import Image, ImageDraw

documentName = "C:/Users/hrint/Documents/Python2022/healthcare/extraction/Referral_Form_page-0001.jpg"

img_file = open(documentName, 'rb') 

image = Image.open(img_file)
width, height =image.size    

In [104]:
def ShowBoundingBox(draw,box,width,height,boxColor):
             
    left = width * box['Left']
    top = height * box['Top'] 
    draw.rectangle([left,top, left + (width * box['Width']), top +(height * box['Height'])],outline=boxColor)   

def ShowSelectedElement(draw,box,width,height,boxColor):
             
    left = width * box['Left']
    top = height * box['Top'] 
    draw.rectangle([left,top, left + (width * box['Width']), top +(height * box['Height'])],fill=boxColor)  


for block in blocks:
#         DisplayBlockInformation(block)    
        draw=ImageDraw.Draw(image)

        # Draw bounding boxes for different detected response objects
        if block['BlockType'] == "KEY_VALUE_SET":
            if block['EntityTypes'][0] == "KEY":
                ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height,'red')
                

            else:
                
                ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height,'green')  
                
        if block['BlockType'] == 'TABLE':
            
            ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height, 'blue')
            
        if block['BlockType'] == 'CELL':
            ShowBoundingBox(draw, block['Geometry']['BoundingBox'],width,height, 'yellow')
            
        if block['BlockType'] == 'SELECTION_ELEMENT':
            if block['SelectionStatus'] =='SELECTED':
                ShowSelectedElement(draw, block['Geometry']['BoundingBox'],width,height, 'blue')    
            
    # Display the image
image.show()


In [82]:
doc = Document(response)
for page in doc.pages:
    print("Fields:")
    for field in page.form.fields:
        print("{} : {}".format(field.key, field.value))

Fields:
Phone: : (432) 432-4233
Referred On: : 01/07/2022
Phone: : (234)324-3244
City, St Zip: : HICKORY, NC 28602-4031
City, St Zip: : Naples, FL 34109
Referred To: : BROWN, VIRGINIA
Fax: : (234) 234-2343
Policy#: : 4555533
Address: : 2406 CENTURY PL SE
DOB: : 10-02-1972
City, St Zip: : Fort Lauderdale, FL 33305
Address: : 2430 Vanderbilt Beach Road, Suite 355
Referred By: : Family Frank
Patient Name: : Sparling, Michael
Phone: : (828) 324-9595
Fax: : (828) 324-4154
Primary/Billing Diagnosis: : /27.21 Secondary pulmonary arterial hypertension
Address: : 3128 Rocket Drive
Insurance: : Affinity Health Plan Medicare
Referral Reason: : None
Attachments: : Unsigned Chart Note (Jan 07, 2022 04:00 PM) (Unsigned Chart Note (Jan 07, 2022 04:00 PM))  Michael_Sparling_1304168_202217142458.xml (Michael_Sparling_1304168_202217142458)


In [103]:
for page in doc.pages:
     # Print tables
    for table in page.tables:
        for r, row in enumerate(table.rows):
            for c, cell in enumerate(row.cells):
                print("Table[{}][{}] = {}".format(r, c, cell.text))
#                 print(dir(cell.text))

Table[0][0] = Patient Name: 
Table[0][1] = Sparling, Michael 
Table[1][0] = Address: 
Table[1][1] = 3128 Rocket Drive 
Table[2][0] = City, St Zip: 
Table[2][1] = Fort Lauderdale, FL 33305 
Table[0][0] = DOB: 
Table[0][1] = 10-02-1972 
Table[1][0] = Phone: 
Table[1][1] = (234)324-3244 
Table[0][0] = Referred To: 
Table[0][1] = BROWN, VIRGINIA 
Table[0][2] = Referred On: 
Table[0][3] = 01/07/2022 
Table[1][0] = Address: 
Table[1][1] = 2406 CENTURY PL SE 
Table[1][2] = Phone: 
Table[1][3] = (828) 324-9595 
Table[2][0] = City, St Zip: 
Table[2][1] = HICKORY, NC 28602-4031 
Table[2][2] = Fax: 
Table[2][3] = (828) 324-4154 
Table[3][0] = Referred By: 
Table[3][1] = Family Frank 
Table[3][2] = 
Table[3][3] = 
Table[4][0] = Address: 
Table[4][1] = 2430 Vanderbilt Beach Road, 
Table[4][2] = Suite 355 Phone: 
Table[4][3] = (432) 432-4233 
Table[5][0] = City, St Zip: 
Table[5][1] = Naples, FL 34109 
Table[5][2] = Fax: 
Table[5][3] = (234) 234-2343 


In [119]:
blocks=response['Blocks']
# print(blocks)
for block in blocks:
    if block['BlockType'] == 'CELL':
        print(block)

{'BlockType': 'CELL', 'Confidence': 93.24407958984375, 'RowIndex': 1, 'ColumnIndex': 1, 'RowSpan': 1, 'ColumnSpan': 1, 'Geometry': {'BoundingBox': {'Width': 0.15735135972499847, 'Height': 0.020467018708586693, 'Left': 0.0757131576538086, 'Top': 0.21685491502285004}, 'Polygon': [{'X': 0.0757131576538086, 'Y': 0.21685491502285004}, {'X': 0.23306451737880707, 'Y': 0.21685491502285004}, {'X': 0.23306451737880707, 'Y': 0.23732194304466248}, {'X': 0.0757131576538086, 'Y': 0.23732194304466248}]}, 'Id': 'c4036aca-4fda-4f9d-bd5b-69b3f06bc23f', 'Relationships': [{'Type': 'CHILD', 'Ids': ['e66fbceb-adea-42ad-b7cb-ee2de278b60f', '127cbb49-a6a7-4545-8be1-8395f46aba88']}]}
{'BlockType': 'CELL', 'Confidence': 93.83567810058594, 'RowIndex': 1, 'ColumnIndex': 2, 'RowSpan': 1, 'ColumnSpan': 1, 'Geometry': {'BoundingBox': {'Width': 0.27439266443252563, 'Height': 0.020467018708586693, 'Left': 0.23306451737880707, 'Top': 0.21685491502285004}, 'Polygon': [{'X': 0.23306451737880707, 'Y': 0.21685491502285004}

In [47]:
print(doc.pages[0].tables[1].rows[0].cells[0])

DOB: 


In [23]:
def map_blocks(blocks, block_type):
    return {
        block['Id']: block
        for block in blocks
        if block['BlockType'] == block_type
    }

blocks = response['Blocks']
tables = map_blocks(blocks, 'TABLE')
cells = map_blocks(blocks, 'CELL')
words = map_blocks(blocks, 'WORD')
selections = map_blocks(blocks, 'SELECTION_ELEMENT')


def get_children_ids(block):
    for rels in block.get('Relationships', []):
        if rels['Type'] == 'CHILD':
            yield from rels['Ids']

In [30]:
import pandas as pd

dataframes = []

for table in tables.values():

    # Determine all the cells that belong to this table
    table_cells = [cells[cell_id] for cell_id in get_children_ids(table)]

    # Determine the table's number of rows and columns
    n_rows = max(cell['RowIndex'] for cell in table_cells)
    n_cols = max(cell['ColumnIndex'] for cell in table_cells)
    content = [[None for _ in range(n_cols)] for _ in range(n_rows)]

    # Fill in each cell
    for cell in table_cells:
        cell_contents = [
            words[child_id]['Text']
            if child_id in words
            else selections[child_id]['SelectionStatus']
            for child_id in get_children_ids(cell)
        ]
        i = cell['RowIndex'] - 1
        j = cell['ColumnIndex'] - 1
        content[i][j] = ' '.join(cell_contents)

    # We assume that the first row corresponds to the column names
    dataframe = pd.DataFrame(content[1:], columns=content[0])
    dataframes.append(dataframe)

In [32]:
len(dataframe)

5

In [37]:
dataframe.head(5)

,Referred To:,"BROWN, VIRGINIA",Referred On:,01/07/2022
0,Address:,2406 CENTURY PL SE,Phone:,(828) 324-9595
1,"City, St Zip:","HICKORY, NC 28602-4031",Fax:,(828) 324-4154
2,Referred By:,Family Frank,,
3,Address:,"2430 Vanderbilt Beach Road,",Suite 355 Phone:,(432) 432-4233
4,"City, St Zip:","Naples, FL 34109",Fax:,(234) 234-2343


In [96]:
def Fibonacci(n):
	if n < 0: print("Incorrect input")
	elif n == 0: return 0
	elif n == 1 or n == 2: return 1
	else: return Fibonacci(n-1) + Fibonacci(n-2)

for i in range(nterms):  
        print(FibRecursion(i), end=", ")

0, 1, 1, 2, 3, 5, 8, 13, 21, 

In [5]:
from trp import Document
import boto3

def get_kv_map():
    # Document
    documentName = "C:/Users/hrint/Documents/Python2022/healthcare/extraction/Referral_Form_page-0001.jpg"
    # Amazon Textract client
    textract = boto3.client('textract')
    # Call Amazon Textract
    with open(documentName, "rb") as document:
        response = textract.analyze_document(
            Document={
                'Bytes': document.read(),
            },
            FeatureTypes=["FORMS", ])

    # Get the text blocks
    blocks=response['Blocks']
    

    # get key and value maps
    key_map = {}
    value_map = {}
    block_map = {}
    for block in blocks:
        block_id = block['Id']
        block_map[block_id] = block
        if block['BlockType'] == "KEY_VALUE_SET":
            if 'KEY' in block['EntityTypes']:
                key_map[block_id] = block
            else:
                value_map[block_id] = block

    return key_map, value_map, block_map

In [7]:
get_kv_map()

({'da119c38-fe26-4b10-961f-9c9b44588429': {'BlockType': 'KEY_VALUE_SET',
   'Confidence': 99.5,
   'Geometry': {'BoundingBox': {'Width': 0.06394796073436737,
     'Height': 0.012023091316223145,
     'Left': 0.6530346870422363,
     'Top': 0.45183175802230835},
    'Polygon': [{'X': 0.6530346870422363, 'Y': 0.45183175802230835},
     {'X': 0.7169826626777649, 'Y': 0.45183175802230835},
     {'X': 0.7169826626777649, 'Y': 0.4638548493385315},
     {'X': 0.6530346870422363, 'Y': 0.4638548493385315}]},
   'Id': 'da119c38-fe26-4b10-961f-9c9b44588429',
   'Relationships': [{'Type': 'VALUE',
     'Ids': ['b2e0c987-c60e-4ff6-9c15-51f48d4c1b76']},
    {'Type': 'CHILD', 'Ids': ['481be931-1a38-41c0-a6d6-338c01d84429']}],
   'EntityTypes': ['KEY']},
  '58c5fd94-25c6-428e-8864-cd10ff420991': {'BlockType': 'KEY_VALUE_SET',
   'Confidence': 99.0,
   'Geometry': {'BoundingBox': {'Width': 0.11740200966596603,
     'Height': 0.012627982534468174,
     'Left': 0.5564625263214111,
     'Top': 0.334639132

In [8]:
def get_kv_relationship(key_map, value_map, block_map):
    kvs = {}
    for block_id, key_block in key_map.items():
        value_block = find_value_block(key_block, value_map)
        key = get_text(key_block, block_map)
        val = get_text(value_block, block_map)
        kvs[key] = val
    return kvs


def find_value_block(key_block, value_map):
    for relationship in key_block['Relationships']:
        if relationship['Type'] == 'VALUE':
            for value_id in relationship['Ids']:
                value_block = value_map[value_id]
    return value_block

In [ ]:
import sys
import re
def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] == 'SELECTED':
                            text += 'X'                           
    return text


def print_kvs(kvs):
    for key, value in kvs.items():
        print(key, ":", value)


def search_value(kvs, search_key):
    for key, value in kvs.items():
        if re.search(search_key, key, re.IGNORECASE):
            return value

def main(file_name):

    key_map, value_map, block_map = get_kv_map()

    # Get Key Value relationship
    kvs = get_kv_relationship(key_map, value_map, block_map)
    print("\n\n== FOUND KEY : VALUE pairs ===\n")
    print_kvs(kvs)

    # Start searching a key value
    while input('\n Do you want to search a value for a key? (enter "n" for exit) ') != 'n':
        search_key = input('\n Enter a search key:')
        print('The value is:', search_value(kvs, search_key))

if __name__ == "__main__":
    file_name = sys.argv[1]
    main(file_name)



== FOUND KEY : VALUE pairs ===

Phone:  : (828) 324-9595 
Referred On:  : 01/07/2022 
City, St Zip:  : Fort Lauderdale, FL 33305 
Referred To:  : BROWN, VIRGINIA 
Fax:  : (828) 324-4154 
Policy#:  : 4555533 
Address:  : 3128 Rocket Drive 
DOB:  : 10-02-1972 
Referred By:  : Family Frank 
Patient Name:  : Sparling, Michael 
Primary/Billing Diagnosis:  : /27.21 Secondary pulmonary arterial hypertension 
Insurance:  : Affinity Health Plan Medicare 
Referral Reason:  : 
Attachments:  : Unsigned Chart Note (Jan 07, 2022 04:00 PM) (Unsigned Chart Note (Jan 07, 2022 04:00 PM))  Michael_Sparling_1304168_202217142458.xml (Michael_Sparling_1304168_202217142458) 


In [ ]:
image=Image.open(documentName)
draw=ImageDraw.Draw(image)
width, height =image.size  
bbox = {"Width": 0.905, "Height": 0.1,"Left": 0.04, "Top": 0.21}
ShowBoundingBox(draw, bbox, width, height,'red')
image.show()

In [ ]:
d = {'Width': 0.06394796073436737,
     'Height': 0.012023091316223145,
     'Left': 0.6530346870422363,
     'Top': 0.45183175802230835}



In [ ]:
import webbrowser, os
import json
import boto3
import io
from io import BytesIO
import sys
from pprint import pprint


def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map[child_id]
                if cell['BlockType'] == 'CELL':
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    if row_index not in rows:
                        # create new row
                        rows[row_index] = {}
                        
                    # get the text value
                    rows[row_index][col_index] = get_text(cell, blocks_map)
    return rows


def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] =='SELECTED':
                            text +=  'X '    
    return text


def get_table_csv_results(file_name):

    with open(file_name, 'rb') as file:
        img_test = file.read()
        bytes_test = bytearray(img_test)
        print('Image loaded', file_name)

    # process using image bytes
    # get the results
    client = boto3.client('textract')

    response = client.analyze_document(Document={'Bytes': bytes_test}, FeatureTypes=['TABLES'])

    # Get the text blocks
    blocks=response['Blocks']
    pprint(blocks)

    blocks_map = {}
    table_blocks = []
    for block in blocks:
        blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)

    if len(table_blocks) <= 0:
        return "<b> NO Table FOUND </b>"

    csv = ''
    for index, table in enumerate(table_blocks):
        csv += generate_table_csv(table, blocks_map, index +1)
        csv += '\n\n'

    return csv

def generate_table_csv(table_result, blocks_map, table_index):
    rows = get_rows_columns_map(table_result, blocks_map)

    table_id = 'Table_' + str(table_index)
    
    # get cells.
    csv = 'Table: {0}\n\n'.format(table_id)

    for row_index, cols in rows.items():
        
        for col_index, text in cols.items():
            csv += '{}'.format(text) + ","
        csv += '\n'
        
    csv += '\n\n\n'
    return csv

def main(file_name):
    file_name = "C:/Users/hrint/Documents/Python2022/healthcare/extraction/Referral_Form_page-0001.jpg"
    table_csv = get_table_csv_results(file_name)

    output_file = 'output.csv'

    # replace content
    with open(output_file, "wt") as fout:
        fout.write(table_csv)

    # show the results
    print('CSV OUTPUT FILE: ', output_file)


if __name__ == "__main__":
    file_name = sys.argv[1]
    main(file_name)